In [17]:
!pip install pandas

# Import libraries

In [2]:
import pandas as pd
import numpy as np
import bs4
import sys
import requests

# Get list of university towns

In [16]:
def list_of_university_towns():
    with open(r'C:\Users\ABHISHEK SHARMA\Downloads\University_towns (1).txt', 'r') as reader:
        lines= reader.readlines()
    reader.close()
    
    unidata= pd.DataFrame(columns=('State', 'RegionName'))
    count= 0
    state= ""
    region= ""
    
    for line in lines:
        line= line.rstrip('n')
        line= line.strip()
        if 'edit' in line:
            state= line.split('[')[0]
        else:
            region=line.split('(',1)[0]
            unidata.loc[count]= [state,region]
            count= count+ 1
            
    return unidata

list_of_university_towns()

,State,RegionName
0,Alabama,AuburnÂ
1,Alabama,DothanÂ
2,Alabama,FairfieldÂ
3,Alabama,FlorenceÂ
4,Alabama,HomewoodÂ
...,...,...
828,Wisconsin,WhitewaterÂ
829,Wyoming,LaramieÂ
830,Puerto Rico,"MayagÃ¼ez, Puerto RicoÂ"
831,Puerto Rico,"Ponce, Puerto Rico"


In [4]:
df= pd.read_excel(r'C:\Users\ABHISHEK SHARMA\Downloads\gdplev.xlsx',skiprows=219)
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,1999Q4,9899.4,12877.6,Unnamed: 7
0,NaN,NaN,NaN,NaN,2000Q1,10002.9,12924.2,NaN
1,NaN,NaN,NaN,NaN,2000Q2,10247.7,13160.8,NaN
2,NaN,NaN,NaN,NaN,2000Q3,10319.8,13178.4,NaN
3,NaN,NaN,NaN,NaN,2000Q4,10439.0,13260.5,NaN
4,NaN,NaN,NaN,NaN,2001Q1,10472.9,13222.7,NaN
...,...,...,...,...,...,...,...,...
80,NaN,NaN,NaN,NaN,2020Q1,21561.1,19010.8,NaN
81,NaN,NaN,NaN,NaN,2020Q2,19520.1,17302.5,NaN
82,NaN,NaN,NaN,NaN,2020Q3,21170.3,18596.5,NaN
83,NaN,NaN,NaN,NaN,2020Q4,21494.7,18794.4,NaN


# Clean dataframe

In [5]:
def clean_GDP_data():
    df= pd.read_excel(r'C:\Users\ABHISHEK SHARMA\Downloads\gdplev.xlsx',skiprows=219)
    df= df.rename(columns={'1999Q4':'Quarter',12877.6:'GDP'})
    df= df[['Quarter','GDP']]
    return df
clean_GDP_data().head()

    
    

,Quarter,GDP
0,2000Q1,12924.2
1,2000Q2,13160.8
2,2000Q3,13178.4
3,2000Q4,13260.5
4,2001Q1,13222.7


In [6]:
def recession_start():
    df= clean_GDP_data()
    i=2
    length= len(df['GDP'])
    recession_start= ''
    while i< length:
        if df.GDP[i] < df.GDP[i-1] and df.GDP[i-1] < df.GDP[i-2]:
            recession_start= df.Quarter[i-1]
            break 
        i+=1
    return recession_start
recession_start()

'2008Q3'

In [11]:
def recession_bottom():
    df= pd.read_excel(r'C:\Users\ABHISHEK SHARMA\Downloads\gdplev.xlsx',skiprows=219)
    df= df.rename(columns={'1999Q4':'Quarter',12877.6:'GDP'})
    df= df[['Quarter','GDP']]
                  
    i=2
    length= len(df['GDP'])
    in_recession= False
    recession_bottom=''
    lowest_gdp= 1000000000
    while i< length:
        if df.GDP[i] < df.GDP[i-1] and df.GDP[i-1] < df.GDP[i-2]:
                  in_recession= True
        if in_recession== True and df.GDP[i] < lowest_gdp:
                lowest_gdp= df.GDP[i]
                recession_bottom= df.Quarter[i]
        if in_recession== True and df.GDP[i] > df.GDP[i-1] and df.GDP[i-1] > df.GDP[i-2]:
                  in_recession= False
        i+=1
    return recession_bottom
recession_bottom()                  

'2009Q2'

## Convert house data to quarters

In [8]:
def convert_house_data_to_quarters():
    states= {
        'AK':'Alaska','AL':'Alabama','AR':'Arkansas','AS':'American Samoa','AZ':'Arizona','CA':'California','CO':'Colorado','CT':'Connecticut','DC':'District of Columbia','DE':'Delaware','FL':'Florida','GA'
             :'Georgia','GU':'Guam','HI':'Hawaii','IA':'Iowa','ID':'Idaho','IL':'Illinois','IN':'Indiana','KS':'Kansas','KY':'Kentucky','LA':'Louisiana','MA':'Massachusetts','MD':'Maryland','ME':'Maine','MI':'Michigan','MN':'Minnesota','MO':'Missouri','MP':'Northern Mariana Islands','MS':'Mississippi','MT':'Montana','NA':'National','NC':'North Carolina'
    }
    
    house= pd.read_csv(r'C:\Users\ABHISHEK SHARMA\Downloads\City_Zhvi_AllHomes.csv')
    house['State'].replace(states,inplace= True)
    house= house.set_index(["State","RegionName"])
    house= house.iloc[:,49:250]
    
    def quarters(col):
        if col.endswith(("01","02","03")):
            s= col[:4]+ "q1"
        elif col.endswith(("04","05","06")):
            s= col[:4]+ "q2"
        elif col.endswith(("07","08","09")):
            s= col[:4]+ "q3"
        else:
            s= col[:4]+ "q4"
        return s
    
    print(house)
    house= house.groupby(quarters,axis=1).mean()
    print(house.groupby(quarters,axis=1).mean())
    house= house.sort_index()
    return house
                
        

convert_house_data_to_quarters()     

                         1999-08-31  1999-09-30  1999-10-31  1999-11-30  \
State       RegionName                                                    
NY          New York       224705.0    226911.0    228704.0    230438.0   
California  Los Angeles    227833.0    228984.0    230138.0    231957.0   
TX          Houston        105728.0    106193.0    106598.0    107025.0   
Illinois    Chicago        161674.0    162512.0    163387.0    164519.0   
TX          San Antonio    102525.0    102869.0    103193.0    103574.0   
...                             ...         ...         ...         ...   
Minnesota   Winton              NaN         NaN         NaN         NaN   
Mississippi Eastabuchie         NaN         NaN         NaN         NaN   
TX          Dean                NaN         NaN         NaN         NaN   
Georgia     Pulaski             NaN         NaN         NaN         NaN   
PA          New Paris           NaN         NaN         NaN         NaN   

                        

1999q4         2000q4         2001q4         2002q4  \
State   RegionName                                                          
Alabama Abbeville    47568.8   49173.666667   51205.333333   54767.583333   
        Adamsville   67205.8   69861.333333   71605.333333   73529.666667   
        Addison      65977.2   65180.333333   70034.333333   77223.000000   
        Akron            NaN            NaN            NaN            NaN   
        Alabaster   137687.8  140043.666667  142850.833333  146390.916667   
...                      ...            ...            ...            ...   
WY      Wheatland        NaN            NaN            NaN            NaN   
        Wilson           NaN            NaN            NaN            NaN   
        Worland          NaN            NaN            NaN            NaN   
        Wright           NaN            NaN            NaN            NaN   
        Yoder            NaN            NaN            NaN            NaN   

                           2003q4         2004q4         2005q4  \
State   RegionName                                                
Alabama Abbeville    59649.666667   64456.500000   70338.666667   
        Adamsville   74643.500000   77587.166667   82324.250000   
        Addison      81260.833333   83947.666667   91953.666667   
        Akron                 NaN   56156.000000   63908.000000   
        Alabaster   149817.000000  155345.250000  162057.583333   
...                           ...            ...            ...   
WY      Wheatland             NaN            NaN  116398.888889   
        Wilson                NaN            NaN  809505.500000   
        Worland               NaN            NaN  113450.333333   
        Wright                NaN            NaN  180784.727273   
        Yoder                 NaN            NaN            NaN   

                           2006q4        2007q4        2008q4        2009q4  \
State   RegionName                                                            
Alabama Abbeville    76386.166667  8.098833e+04  8.346125e+04  8.392067e+04   
        Adamsville   86701.333333  9.316717e+04  8.814592e+04  7.940200e+04   
        Addison      99892.833333  1.076549e+05  1.066039e+05  9.793442e+04   
        Akron        70061.666667  7.033550e+04  6.791992e+04  5.859733e+04   
        Alabaster   173053.833333  1.775070e+05  1.717838e+05  1.637992e+05   
...                           ...           ...           ...           ...   
WY      Wheatland   132801.916667  1.365953e+05  1.391387e+05  1.417987e+05   
        Wilson      999670.250000  1.084756e+06  1.092029e+06  1.055497e+06   
        Worland     120530.666667  1.286812e+05  1.329652e+05  1.354898e+05   
        Wright      209345.166667  2.109474e+05  2.125706e+05  2.095042e+05   
        Yoder                 NaN           NaN  1.552416e+05  1.510840e+05   

                           2010q4         2011q4         2012q4  \
State   RegionName                                                
Alabama Abbeville    84710.250000   79271.333333   78588.250000   
        Adamsville   77138.666667   74243.333333   66537.416667   
        Addison      91275.250000   88388.000000   86155.416667   
        Akron        53457.416667   51660.166667   56226.000000   
        Alabaster   160566.000000  151540.166667  149915.250000   
...                           ...            ...            ...   
WY      Wheatland   141144.583333  147012.083333  149063.333333   
        Wilson      980667.166667  911948.500000  978975.083333   
        Worland     134067.250000  134587.916667  136381.083333   
        Wright      204893.333333  200651.250000  206225.666667   
        Yoder       150290.666667  151046.833333  158652.083333   

                           2013q4        2014q4        2015q4      2016q4  
State   RegionName                                                         
Alabama Abbeville    77234.583333  7.781442e+04  8.012017e+04    81612.25  
        Adamsville  

In [9]:
unitowns= get_list_of_university_towns()
unitowns= unitowns.set_index(['State','RegionName'])
housingdata= convert_house_data_to_quarters() 

housingdata['2008q2']= housingdata['2008q2']/housingdata['2009q2']
housingdata= housingdata['2008q2'].to_frame()
housingdata= housingdata.dropna()

                         1999-08-31  1999-09-30  1999-10-31  1999-11-30  \
State       RegionName                                                    
NY          New York       224705.0    226911.0    228704.0    230438.0   
California  Los Angeles    227833.0    228984.0    230138.0    231957.0   
TX          Houston        105728.0    106193.0    106598.0    107025.0   
Illinois    Chicago        161674.0    162512.0    163387.0    164519.0   
TX          San Antonio    102525.0    102869.0    103193.0    103574.0   
...                             ...         ...         ...         ...   
Minnesota   Winton              NaN         NaN         NaN         NaN   
Mississippi Eastabuchie         NaN         NaN         NaN         NaN   
TX          Dean                NaN         NaN         NaN         NaN   
Georgia     Pulaski             NaN         NaN         NaN         NaN   
PA          New Paris           NaN         NaN         NaN         NaN   

                        

KeyError: '2008q2'

## Run t-test

In [15]:
def run_ttest():
    gt= get_list_of_university_towns()
    dt= convert_house_data_to_quarters()
    gt= gt.set_index(['State','RegionName'])
    good= ''
    
    list= dt.index.list(gt.index).astype(bool)
    dt.insert(loc=0, column='univ',value=list)
    
    univ_town= dt[dt['univ']== True]
    non_univ_town= dt[dt['univ']== False]
    
    univ_ratio= univ_town[recession_start()]/ univ_town[recession_bottom()]
    non_univ_ratio=  non_univ_town[recession_start()]/ non_univ_town[recession_bottom()]
    p_value= ttest_ind(univ_ratio.dropna(), non_univ_ratio.dropna())[1]
    difference= p_value< .01
    
    if univ_ratio.mean()< non_univ_ratio.mean():
        good= 'university town'
        
    else:
            good= 'non university town'
            
    return (difference,p_value, good)

run_ttest

<function __main__.run_ttest()>